Ramp Score:

This is a preprocessing file.


In this file, there are the following methods:
* To convert the json file to a csv.
* To convert the point co-ordinates to latitude and longitude.
* To get address from given latitide and longitude.

Note: Finding the address for a given latitide and longitude for an entire document is a time consuming process.

In [2]:
import json
import pandas as pd

with open("../data/JSON_files/RampsNew/RampsNew.json",'r') as f:
    data = json.loads(f.read())
  
df = pd.json_normalize(
    data, 
    record_path =['features']
)

In [3]:
df.head

<bound method NDFrame.head of           type     id geometry.type                      geometry.coordinates  \
0      Feature      0         Point   [-7911843.053800002, 5213855.430799998]   
1      Feature      1         Point   [-7912104.434000002, 5213764.354100004]   
2      Feature      2         Point  [-7912111.2201999985, 5213780.467799999]   
3      Feature      3         Point    [-7912093.786999997, 5213788.37470001]   
4      Feature      4         Point   [-7911853.312899999, 5213875.931500003]   
...        ...    ...           ...                                       ...   
26207  Feature  26207         Point        [-7908859.5842, 5213906.350699998]   
26208  Feature  26208         Point        [-7908620.2472, 5213876.224299997]   
26209  Feature  26209         Point              [-7908621.175, 5213877.0612]   
26210  Feature  26210         Point        [-7908602.0033, 5213865.763899997]   
26211  Feature  26211         Point        [-7910820.3533, 5209827.192599997]  

In [4]:
df.head

<bound method NDFrame.head of           type     id geometry.type                      geometry.coordinates  \
0      Feature      0         Point   [-7911843.053800002, 5213855.430799998]   
1      Feature      1         Point   [-7912104.434000002, 5213764.354100004]   
2      Feature      2         Point  [-7912111.2201999985, 5213780.467799999]   
3      Feature      3         Point    [-7912093.786999997, 5213788.37470001]   
4      Feature      4         Point   [-7911853.312899999, 5213875.931500003]   
...        ...    ...           ...                                       ...   
26207  Feature  26207         Point        [-7908859.5842, 5213906.350699998]   
26208  Feature  26208         Point        [-7908620.2472, 5213876.224299997]   
26209  Feature  26209         Point              [-7908621.175, 5213877.0612]   
26210  Feature  26210         Point        [-7908602.0033, 5213865.763899997]   
26211  Feature  26211         Point        [-7910820.3533, 5209827.192599997]  

Transformation of co-ordiantes to latitude and longitude

In [2]:
from pyproj import Transformer

# Define the projection system of the input coordinates
in_proj = 'epsg:3857' # Assumes input coordinates are in Web Mercator projection

# Define the projection system of the output coordinates
out_proj = 'epsg:4326' # Assumes output coordinates are in WGS84 projection (latitude and longitude)

# Define the transformer object to convert from input to output projection system
transformer = Transformer.from_crs(in_proj, out_proj)

# Define the input coordinates

x=-7911843.053800002
y=5213855.430799998

# Convert the input coordinates to latitude and longitude
lat, lon = transformer.transform(x, y)

# Print the latitude and longitude
print(f"Latitude: {lat}, Longitude: {lon}")

def transform_coordinates(co_ordinate):
    x = co_ordinate[0]
    y = co_ordinate[1]
    return transformer.transform(x,y)


df['latitude'] = df['geometry.coordinates'].apply(lambda x: transform_coordinates(x)[0]).astype(float)
df['longitude'] = df['geometry.coordinates'].apply(lambda x: transform_coordinates(x)[1]).astype(float)

Latitude: 42.35200917969154, Longitude: -71.0732954078341


In [5]:
df.to_csv("../data/Ramp_score.csv", index=False)

In [15]:
df[df['latitude']==42.31219332075908]

,type,id,geometry.type,geometry.coordinates,properties.FID,properties.ObjectID_1,properties.ObjectID_2,properties.CREEC_Type,properties.CREEC_Posi,properties.RAMP_MATL,...,properties.NEAR_DIST,properties.FID_1,properties.DESCRIPTIO,properties.INT_ID,properties.Failure_Ty,properties.score_1,properties.Fail_Label,properties.RampID_1,latitude,longitude
11024,Feature,11024,Point,"[-7915300.1722, 5207859.816400006]",11024,11977,11977,Diagonal,Diagonal,Portland Cement Concrete,...,13.010335,9232,Brookside Ave / Cornwall St,1417,Pvt/Cswk Failure,95.8,Gutter x-slope 4.8%; Crosswalk misaligned;,1417.1,42.312193,-71.104351


Finding the address given the latitude and longitude

In [ ]:
async def get_response(zipcodes):
    return await asyncio.gather(**zipcodes)

zipcodes = []

for index,row in coord.iterrows():
    zipcodes.append(asyncio.ensure_future(get_zipcode(row, geolocator,lat_field='y', lon_field='x')))

#Wait till all the threads are finished running and get the data
#Takes a long time to runn
response = asyncio.run(get_response(zipcodes))
df['address'] = response
# df['pincode'] = df['address'].apply(lambda x: x['postcode'])


In [19]:
#Generate the address using the latitude and longitude
import geopy
import pandas as pd
geolocator = geopy.Nominatim(user_agent='my-application1')
import asyncio 



zipcodes = []

count = 0


async def get_zipcode(df, geolocator, lat_field, lon_field):
    location = geolocator.reverse((df[lat_field], df[lon_field]))
    return location.raw['address']
for index,row in df.iterrows():
    zipcodes.append(asyncio.ensure_future(get_zipcode(row, geolocator,lat_field='latitude', lon_field='longitude')))

response = await asyncio.gather(*zipcodes)
response

df["road"] = [i.get("road") for i in response]
df["county"] = [i.get("county") for i in response]
df["neighbourhood"] = [i.get("neighbourhood") for i in response]
df["zipcodes"] = [i.get("postcode") for i in response]

[{'shop': 'Brooks Brothers',
  'house_number': '46',
  'road': 'Newbury Street',
  'commercial': 'Newbury Street',
  'suburb': 'Back Bay',
  'city': 'Boston',
  'county': 'Suffolk County',
  'state': 'Massachusetts',
  'ISO3166-2-lvl4': 'US-MA',
  'postcode': '02116',
  'country': 'United States',
  'country_code': 'us'},
 {'house_number': '110',
  'road': 'Newbury Street',
  'commercial': 'Newbury Street',
  'suburb': 'Back Bay',
  'city': 'Boston',
  'county': 'Suffolk County',
  'state': 'Massachusetts',
  'ISO3166-2-lvl4': 'US-MA',
  'postcode': '02116',
  'country': 'United States',
  'country_code': 'us'},
 {'shop': 'Cole Haan',
  'house_number': '109',
  'road': 'Newbury Street',
  'commercial': 'Newbury Street',
  'suburb': 'Back Bay',
  'city': 'Boston',
  'county': 'Suffolk County',
  'state': 'Massachusetts',
  'ISO3166-2-lvl4': 'US-MA',
  'postcode': '02116',
  'country': 'United States',
  'country_code': 'us'},
 {'house_number': '103-105',
  'road': 'Newbury Street',
  'c